In [ ]:
# Instalar las librerías
!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

Looking in indexes: https://test.pypi.org/simple/


In [ ]:
from nbpep8.nbpep8 import pep8

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import urllib.parse

# Datos de ejemplo de productos
productos = [
    {"id": 1, "nombre": "Cuaderno", "precio": 10.00},
    {"id": 2, "nombre": "Lápiz", "precio": 0.80},
    {"id": 3, "nombre": "Borrador", "precio": 1.00},
    {"id": 4, "nombre": "Tijeras", "precio": 3.00},
    {"id": 5, "nombre": "Block tamaño carta", "precio": 4.00},
    {"id": 6, "nombre": "Cartulina", "precio": 1.00},
    {"id": 7, "nombre": "Diccionario", "precio": 15.00},
]


class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    """Clase para manejar las solicitudes HTTP entrantes."""

    def _set_headers(self, status_code=200, content_type='text/html'):
        """Establece las cabeceras de la respuesta HTTP."""
        self.send_response(status_code)
        self.send_header('Content-type', content_type)
        self.end_headers()

    def do_GET(self):
        """Maneja las solicitudes GET."""
        if self.path == '/':
            # Si la ruta es '/', devuelve un mensaje de saludo.
            self._set_headers()
            self.wfile.write(b'Hello, world!')
        elif self.path == '/productos':
            # Si la ruta es '/productos', devuelve la lista de productos en formato JSON.
            self._set_headers(content_type='application/json')
            self.wfile.write(json.dumps(productos).encode('utf-8'))
        else:
            # Si la ruta no se encuentra, devuelve un mensaje de error.
            self._set_headers(status_code=404)
            self.wfile.write(b'Not found')

    def do_POST(self):
        """Maneja las solicitudes POST."""
        if self.path == '/productos':
            # Si la ruta es '/productos', procesa la solicitud para agregar un nuevo producto.
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            post_data = post_data.decode('utf-8')
            parsed_data = urllib.parse.parse_qs(post_data)
            nuevo_producto = {
                'nombre': parsed_data.get('nombre', [''])[0],
                'precio': parsed_data.get('precio', [''])[0]
            }
            if nuevo_producto['nombre'] and nuevo_producto['precio']:
                """ Si se proporcionan datos válidos para el nuevo producto,
                lo agrega a la lista."""
                nuevo_producto['precio'] = float(nuevo_producto['precio'])
                productos.append(nuevo_producto)
                self._set_headers(status_code=201, content_type=
                                  'application/json')
                self.wfile.write(json.dumps({"mensaje":
                            "Producto agregado correctamente"}).encode('utf-8'))
            else:
                # Si faltan datos o son incorrectos, devuelve un mensaje de error.
                self._set_headers(status_code=400, content_type=
                                  'application/json')
                self.wfile.write(json.dumps({"error":
                              "Datos de producto incorrectos"}).encode('utf-8'))
        else:
            # Si la ruta no se encuentra, devuelve un mensaje de error.
            self._set_headers(status_code=404, content_type='application/json')
            self.wfile.write(json.dumps({"error":
                                         "Ruta no encontrada"}).encode('utf-8'))


def run(server_class=HTTPServer, handler_class=SimpleHTTPRequestHandler,
        port=8001):
    """Inicia el servidor HTTP."""
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f'Starting server on port {port}...')
    httpd.serve_forever()


if __name__ == '__main__':
    run(port=8001)


pep8(_ih)